In [ ]:
import spacy
finding_sim.py
no viors ong
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile
model = KeyedVectors. load_word2vec_format (word_vectors/word2ve_hp_ner_model.txt', binary=False)
"""
SOURCE:
https://stackoverflow.com/questions/54717449/mapping-word-vector-to-the-most-similar-closest-word-using-spacy
"""
def spacy_similarity(your_word):
nip = spacy. load ("en_core_web_1g")
ms = nip. vocab.vectors.most_similar(
p.asarray([nip. vocab.vectors[nip.vocab.strings[your_word]]]), n=10)
words = [nlp.vocab.strings[w] for w in ms [0][0]]
distances = ms [2]
print (words)